<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>

# AutoGen Orchestrator Agent

In this tutorial, we'll explore orchestrator agent workflows with [AutoGen GroupChats](https://microsoft.github.io/autogen/dev//user-guide/core-user-guide/design-patterns/group-chat.html).

**Agent orchestration** enables collaboration among multiple specialized agents, activating only the most relevant one based on the current subtask context. Instead of relying on a fixed sequence, agents dynamically participate depending on the state of the conversation. At termination, results are synthesized together.

Agent orchestrator workflows simplifies this routing pattern through a central orchestrator (`GroupChatManager`) that selectively delegates tasks to the appropriate agents. Each agent monitors the conversation but only contributes when their specific expertise is required. With Phoenix tracing, you get full visibility into the orchestration flow to see which agents engaged, when they were activated, and why.

In this example, we'll build a smart trip planning assistant where subtasks like destination research, hotel booking, and activity suggestions are dynamically sent to the right specialized agent.

By the end of this tutorial, you’ll learn how to:

- Set up multiple specialized AutoGen agents in a `GroupChat`

- Use a `GroupChatManager` to enable dynamic agent routing

- Incorporate human feedback in your agent set up

- Trace and visualize agent interactions using Phoenix

⚠️ You'll need an OpenAI Key for this tutorial.


## Set up Keys and Dependencies


In [ ]:
!pip install -qq pyautogen==0.9 autogen-agentchat~=0.2

In [ ]:
!pip install -qqq arize-phoenix arize-phoenix-otel openinference-instrumentation-openai

In [ ]:
import os
from getpass import getpass

import autogen

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")

os.environ["OPENAI_API_KEY"] = openai_api_key

In [ ]:
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"
if not os.environ.get("PHOENIX_CLIENT_HEADERS"):
    os.environ["PHOENIX_CLIENT_HEADERS"] = "api_key=" + getpass("Enter your Phoenix API key: ")

## Configure Tracing


In [ ]:
from phoenix.otel import register

tracer_provider = register(
    project_name="autogen-agents",
    endpoint="https://app.phoenix.arize.com/v1/traces",
    auto_instrument=True,
)

## Example Orchestrator Task: Travel Agent

This example shows how to build a dynamic travel planning assistant. A `GroupChatManager` coordinates specialized agents to adapt to the user's evolving travel needs.

**User Interaction**:
A `UserProxyAgent` acts as the human user, configured with `human_input_mode="TERMINATE" `and a custom `is_termination_msg` that ends the session when a message ends with TERMINATE.

**Specialized Travel Agents**:
Three AssistantAgents handle specific tasks.

- Flight Planner — suggests flight options.

- Hotel Finder — recommends accommodations.

- Activity Suggester — proposes activities and attractions.

**GroupChat Setup**:
A GroupChat bundles the user and specialized agents, managing message flow with a maximum round limit (ex: 10 rounds).

**Orchestrator**:
The `GroupChatManager` oversees the conversation, routing tasks to the right agent based on context.

![Diagram](https://storage.googleapis.com/arize-phoenix-assets/assets/images/autogen_orchestrator_diagram.png)

## Define Agent

The `llm_config` specifies the configuration used for all the assistant agents.


In [ ]:
llm_config = {
    "model": "gpt-4o",
    "api_key": os.environ["OPENAI_API_KEY"],
}

In [ ]:
# Specialized LLM Agents
flight_planner = autogen.AssistantAgent(
    name="FlightPlanner",
    llm_config=llm_config,
    system_message="You are a flight planning assistant. You help book flights and find the best travel routes. Focus on using freely accessible sources.",
)

hotel_finder = autogen.AssistantAgent(
    name="HotelFinder",
    llm_config=llm_config,
    system_message="You are a hotel booking assistant. You help find the best accommodations. Focus on using freely accessible sources.",
)

activity_suggester = autogen.AssistantAgent(
    name="ActivitySuggester",
    llm_config=llm_config,
    system_message="You are a travel activity expert. You suggest interesting activities and tours in a destination.",
)

In [ ]:
user_proxy = autogen.UserProxyAgent(
    name="UserProxy",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=5,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
    system_message="A human user seeking travel planning assistance. Reply TERMINATE when the task is done.",
)

agents = [user_proxy, flight_planner, hotel_finder, activity_suggester]

group_chat = autogen.GroupChat(agents=agents, messages=[], max_round=10)

manager = autogen.GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config,
    system_message="""
    You are a coordinator managing a travel planning discussion between a user, a flight planner, a hotel finder, and an activity suggester.
    Your goal is to ensure the user's request is fully addressed by coordinating the specialists.
    Ensure each specialist contributes relevant information sequentially (e.g., flights first, then hotels, then activities, unless the user specifies otherwise).
    Summarize the final plan. Be sure to reply TERMINATE when the plan is complete.
    """,
)

## Run Agent

In [ ]:
from opentelemetry.trace import StatusCode

tracer = tracer_provider.get_tracer(__name__)
with tracer.start_as_current_span(
    "TravelAgent",
    openinference_span_kind="agent",
) as agent_span:
    agent_span.set_status(StatusCode.OK)
    user_proxy.initiate_chat(
        manager,
        message="I want to plan a 5-day trip to Paris, France, sometime in October. I'm interested in museums and good food. Find flight options from SFO, suggest mid-range hotels near the city center, and recommend some relevant activities.",
    )

## View Results in Phoenix

When viewing the traces in Phoenix, you can see how the `GroupChatManager` delegated subtasks to specialized agents step-by-step. The trace shows the order in which each agent responded, making it easy to verify the flow from flight planning to hotel booking to activity suggestions.

![Results](https://storage.googleapis.com/arize-phoenix-assets/assets/images/autogen_routing_results.png)


Run the cell below to see full tracing results.

In [ ]:
from IPython.display import HTML

HTML("""
<video width="800" height="600" controls>
  <source src="https://storage.googleapis.com/arize-phoenix-assets/assets/images/autogen_router_agent.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")